In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import plotly.express as px

from crud_mongodb import AnimalShelter

###########################
# Data Manipulation / Model
###########################

db = AnimalShelter() 

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                 style={'height': '80px', 'margin-right': '20px'}),
        html.H1('Grazioso Salvare Dashboard - Sarah Wagner',
                style={'margin': 'auto'})
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}),
    html.Hr(),
    
    # Interactive filtering options
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All Animals', 'value': 'all'},
            {'label': 'Dogs', 'value': 'dog'},
            {'label': 'Cats', 'value': 'cat'},
            {'label': 'Other', 'value': 'other'},
            {'label': 'Water Rescue', 'value': 'water_rescue'},
            {'label': 'Mountain Rescue', 'value': 'mountain_rescue'},
            {'label': 'Tracking', 'value': 'tracking'}
        ],
        value='all',
        labelStyle={'display': 'inline-block'}
    ),
    html.Button('Reset', id='reset-button', n_clicks=0),
    html.Br(),
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[],
        data=[],  
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts and Map
    html.Div(className='row', style={'display': 'flex', 'justify-content': 'space-around'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
        html.Div(id='second-chart-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'columns'),
     Output('datatable-id', 'data'),
     Output('graph-id', "children"),
     Output('map-id', "children"),
     Output('second-chart-id', "children")],
    [Input('filter-type', 'value'),
     Input('reset-button', 'n_clicks'),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_dashboard(filter_type, reset_clicks, index):
    query = {}
    if filter_type == 'dog':
        query = {"animal_type": "Dog"}
    elif filter_type == 'cat':
        query = {"animal_type": "Cat"}
    elif filter_type == 'other':
        query = {"animal_type": {"$nin": ["Dog", "Cat"]}}
    elif filter_type == 'water_rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
    elif filter_type == 'mountain_rescue':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}
    elif filter_type == 'tracking':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound"]}}
    
    # Fetch filtered data
    filtered_df = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

    # Create DataTable columns dynamically
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns]
    data = filtered_df.to_dict('records')

    # Show Top 5 Breeds
    graph_component = "No breed data available"
    if 'breed' in filtered_df.columns:
        breed_counts = filtered_df['breed'].value_counts()
        top_n = 5  # Show the top 5 breeds
        if len(breed_counts) > top_n:
            top_breeds = breed_counts.nlargest(top_n)
            other_count = breed_counts.iloc[top_n:].sum()  # Sum remaining breeds as "Other"
            breed_counts = pd.concat([top_breeds, pd.Series({'Other': other_count})])

        fig = px.pie(values=breed_counts.values, names=breed_counts.index, 
                     title='Top 5 Animal Breeds', hole=0.3)
        graph_component = dcc.Graph(figure=fig)

    # Second Chart - Age Distribution
    second_chart = "No age data available"
    if 'age_upon_outcome' in filtered_df.columns:
        fig2 = px.bar(filtered_df, x='age_upon_outcome', title='Age Distribution')
        second_chart = dcc.Graph(figure=fig2)

    # Update Location Based on Selected Row
    row = index[0] if index and len(filtered_df) > 0 else 0
    default_lat, default_lon = 30.75, -97.48

    try:
        lat, lon = float(filtered_df.iloc[row]['location_lat']), float(filtered_df.iloc[row]['location_long'])
    except (IndexError, ValueError, KeyError):
        lat, lon = default_lat, default_lon

    map_component = dl.Map(style={'width': '1000px', 'height': '500px'},
                           center=[lat, lon], zoom=10, children=[
                               dl.TileLayer(id="base-layer-id"),
                               dl.Marker(position=[lat, lon], children=[
                                   dl.Tooltip(filtered_df.iloc[row]['name'] if row < len(filtered_df) else "Unknown"),
                                   dl.Popup([
                                       html.H1("Animal Name"),
                                       html.P("Sarah Wagner Project Two"),
                                       html.P(filtered_df.iloc[row]['name'] if row < len(filtered_df) else "Unknown")
                                   ])
                               ])
                           ])

    return columns, data, graph_component, map_component, second_chart

app.run_server(debug=True)


Dash app running on http://127.0.0.1:14375/
